In [ ]:
# Copyright 2023, Acadential, All rights reserved.

# 13-8. Normalization
1. BatchNorm \
    1-1. BatchNorm1d, BatchNorm2d, BatchNorm3d
2. LayerNorm
3. InstanceNorm \
    2-1. InstanceNorm1d, InstanceNorm2d, InstanceNorm3d
4. GroupNorm

## Import libraries

In [1]:
import numpy as np 
import torch 
from torch import nn 

## Utility functions

### 1D Data Generator

예시 Data을 생성하는 함수를 구현해보도록 하겠습니다.

저희가 생성할 데이터는 shape이 ```(batch_size=16, num_features=8)```인 2D random tensor x입니다.

num_features은 num_channels으로도 해석할 수 있습니다.

Batch Norm이 어떻게 작동하는지 확인하기 위해서 각 feature별로 다음 분포를 따르도록 데이터를 생성해보겠습니다:
- i번째 feature의 ```평균값(mean)```이 (0, 1, 2, ... 8) 
- i번째 feature의 ```표준 분포값(standard deviation)```이 (1, 2, 3, ..., 9)

되도록 구성합니다.

즉, 각 ```x[i][j]```은 ```mean이 j```이고 ```standard deviation이 j+1```인 분포에서 샘플링된 random한 값입니다.



In [2]:
batch_size=16
num_features = 8

In [3]:
def data_generator_1d_vary_wrt_channel():
    x = torch.normal(0, 1, size=(batch_size, num_features))
    for i in range(num_features):
        x[:, i] = x[:, i]*(i+1) + i
    
    return x


# BatchNorm

## BatchNorm1d

## 주의 사항

```참고로 주의할 점은 BatchNorm1d의 forward pass은 1d tensor가 아니라 2d tensor 혹은 3d tensor을 입력값으로 받는다.```

즉, ```shape이 (batch_size, num_features)``` 혹은 ```(batch_size, num_features, num_length)```인 tensor을 입력받습니다.


원래는 num_features 크기의 1D tensor가 batch_size 개수 만큼 묶어서 (batch_size, num_features) shape의 batch로 구성하기 때문입니다.


## Initialize data

In [4]:
x = data_generator_1d_vary_wrt_channel()

In [5]:
x.shape

torch.Size([16, 8])

## Check input data statistics

In [6]:
torch.mean(x, axis=0)

tensor([0.0401, 0.6258, 1.2452, 2.7926, 5.3854, 6.5786, 8.0462, 4.3371])

Batch_size에 해당되는 dimension으로 평균을 내면 평균값이 대략적으로 (0, 1, 2, ... 8)이 되는 것을 확인할 수 있습니다.

In [7]:
torch.std(x, axis=0)

tensor([0.8103, 1.5840, 2.3909, 3.9327, 4.6463, 6.1254, 6.0849, 6.9148])

Batch_size에 해당되는 dimension으로 표준편차를 구하면 표준편차가 대략적으로 (1, 2, 3, ... 9)이 되는 것을 확인할 수 있습니다.

## BatchNorm1d Forward pass

처음에 BatchNorm을 initialize할때, ```gamma=0```와 ```beta=1```의 값으로 initialize됩니다.\
그리고, ```affine=True``` 으로 설정할 경우, gamma와 beta값은 trainable해집니다. \
즉, ```affine=True```은 fully connected layer을 batchnorm바로 뒤에 넣어주는 역할인 셈입니다.

즉, batchnorm layer도 ```affine=True```이면 ```trainable한 layer```가 되는 셈입니다.

In [8]:
norm = nn.BatchNorm1d(num_features=num_features,
                     affine=True)

In [9]:
y = norm(x)

## Check statistics of batchnorm1d's output

### Batchnorm 출력값의 평균

In [10]:
torch.mean(y, axis=0)

tensor([ 1.4901e-08, -7.4506e-09,  0.0000e+00, -5.9605e-08,  0.0000e+00,
        -5.9605e-08,  8.1956e-08,  2.9802e-08], grad_fn=<MeanBackward1>)

Batch Norm의 출력값을 (batch size에 해당되는 dimension으로) 평균을 내면 대략적으로 평균값이 (0, 0, 0, ... 0)이 되는 것을 확인할 수 있습니다.

### Batchnorm 출력값의 표준편차

In [11]:
torch.std(y, axis=0)

tensor([1.0328, 1.0328, 1.0328, 1.0328, 1.0328, 1.0328, 1.0328, 1.0328],
       grad_fn=<StdBackward0>)

Batch Norm의 출력값을 (batch size에 해당되는 dimension으로) 표준편차를 계산하면 대략적으로 표준편차가 (1, 1, 1, ... 1)이 되는 것을 확인할 수 있습니다.


### 참고 사항

(```affine==True```인 경우, 학습하는 과정에서 gamma와 beta가 학습되면서 batch norm의 출력값의 평균과 표준편차는 바뀝니다.

참고로, BatchNorm1d forwardpass의 수식에 따르면 Batchnorm의 출력값은 평균 = beta, 표준편차 = gamma 입니다. \
 처음에 Batchnorm이 initialize될때 beta은 0, gamma은 1로 initialize되므로 위와 같이 평균은 0, 표준편차는 1이 되는 것을 확인할 수 있습니다!
 
 

## BatchNorm2d

BatchNorm2d은 일반적으로 CNN layer의 output (batch_size, output_channel_size, height, width) 을 처리할때 사용됩니다.

**참고로 주의할 점은 BatchNorm2d은 인풋으로 4D tensor (batch_size, num_features, height, width)을 받습니다!**

원래는 channel 개수 만큼 쌓은 2D tensor가 batch_size 개수 만큼 묶어서 **(batch_size, channel_size, height, width)** 의 형태로 구성되기 때문입니다.

일반적으로 2D image에 적용되는 CNN에서 사용되는 BatchNorm은 (batch_size, channel_size, heigth, width)을 입력받습니다.\
따라서 2D 이미지에 사용되는 BatchNorm이라 BatchNorm2d라고 불리게 되었고 (이름과는 상관없이 CNN의 출력값인) 4D tensor을 입력받게 된 것으로 보입니다.

CNN -> 2D image에 사용 -> CNN에 사용되는 BatchNorm을 BatchNorm2d로 부르기로 된 것 같습니다.

In [12]:
batch_size=16
num_features = 8
width = 32
height = 32

In [13]:
def data_generator_2d_vary_wrt_channel():
    x = torch.normal(0, 1, size=(batch_size, num_features, width, height))
    for i in range(num_features):
        x[:, i, :, :] = x[:, i, :, :]*(i+1) + i
    
    return x


위 함수는 shape이 ```(batch_size, num_features, width, height)``` 인 random tensor을 생성하는 함수입니다.

각 ```x[i][j][h][w]```은 ```mean이 j```이고 ```standard deviation이 j+1```이도록 샘플링합니다.



## Check input data statistics

In [14]:
x = data_generator_2d_vary_wrt_channel()

In [15]:
x.shape

torch.Size([16, 8, 32, 32])

### 평균 계산
Batch_size, width, heigth에 해당되는 axis에 대해서 평균 계산

In [16]:
torch.mean(x, axis=[0, 2, 3])

tensor([-0.0070,  0.9769,  1.9842,  3.0054,  3.9610,  4.9169,  5.9768,  6.9984])

두번째 dimension에 해당되는 feature들의 평균값들이 대략적으로 (0, 1, 2, ... 8)이 되는 것을 확인할 수 있음. \
```x[i][j][h][w]```의 ```mean == j```

### 표준편차 계산
Batch_size, width, heigth에 해당되는 axis에 대해서 표준편차 계산

In [17]:
torch.std(x, axis=[0, 2, 3])

tensor([1.0007, 2.0141, 2.9779, 4.0197, 5.0077, 5.9681, 6.9464, 7.9319])

두번째 dimension에 해당되는 feature들의 표준편차값들이 대략적으로 (1, 2, 3, ... 9)이 되는 것을 확인할 수 있음. \
```x[i][j][h][w]```의 ```standard deviation == j+1```

## BatchNorm2d Forward pass

In [18]:
norm = nn.BatchNorm2d(num_features=8,
                     affine=True)  ## BatchNorm2d Forward pass

In [19]:
x.shape

torch.Size([16, 8, 32, 32])

In [20]:
y = norm(x)

## Check statistics of batchnorm2d's output

### BatchNorm2d 출력값의 평균

In [21]:
torch.mean(torch.mean(y, axis=[2, 3]))

tensor(-8.5129e-09, grad_fn=<MeanBackward0>)

Batch Norm의 출력값을 (batch size, width, height에 해당되는 dimension들로) 평균을 내면 대략적으로 평균값이 (0, 0, 0, ... 0)이 되는 것을 확인할 수 있습니다.

### BatchNorm2d 출력값의 표준편차

In [22]:
torch.mean(torch.std(y, axis=[2, 3]))

tensor(0.9998, grad_fn=<MeanBackward0>)

Batch Norm의 출력값을 (batch size, width, height에 해당되는 dimension들로) 표준편차를 계산하면 대략적으로 표준편차가 (1, 1, ..., 1)이 되는 것을 확인할 수 있습니다.

## BatchNorm3d

In [23]:
batch_size=16
num_features = 8
depth = 16
width = 32
height = 32

In [24]:
def data_generator_3d_vary_wrt_channel():
    x = torch.normal(0, 1, size=(batch_size, num_features, depth, width, height))
    for i in range(num_features):
        x[:, i, :, :, :] = x[:, i, :, :, :]*(i+1) + i
    
    return x


In [25]:
x = data_generator_3d_vary_wrt_channel()

In [26]:
x.shape

torch.Size([16, 8, 16, 32, 32])

### 평균 계산
Batch_size, depth, width, heigth에 해당되는 axis에 대해서 평균 계산

In [27]:
torch.mean(x, axis=[0, 2, 3, 4])

tensor([-9.7681e-04,  1.0006e+00,  1.9998e+00,  3.0083e+00,  3.9906e+00,
         5.0104e+00,  5.9970e+00,  7.0058e+00])

두번째 dimension에 해당되는 feature들의 평균값들이 대략적으로 (0, 1, 2, ... 8)이 되는 것을 확인할 수 있음. \
```x[i][j][h][w]```의 ```mean == j```

### 표준편차 계산
Batch_size, depth, width, heigth에 해당되는 axis에 대해서 표준편차 계산

In [28]:
torch.std(x, axis=[0, 2, 3, 4])

tensor([0.9983, 1.9976, 3.0002, 4.0063, 4.9901, 5.9976, 7.0137, 8.0031])

두번째 dimension에 해당되는 feature들의 표준편차값들이 대략적으로 (1, 2, 3, ... 9)이 되는 것을 확인할 수 있음. \
```x[i][j][h][w]```의 ```standard deviation == j+1```

## BatchNorm3d Forward pass

In [29]:
norm = nn.BatchNorm3d(num_features=8,
                      affine=True)  ## BatchNorm3d Forward pass

In [30]:
x.shape

torch.Size([16, 8, 16, 32, 32])

In [31]:
y = norm(x)

## Check statistics of batchnorm3d's output

### BatchNorm3d 출력값의 평균

In [32]:
torch.mean(y, axis=[0, 2, 3, 4])

tensor([-7.5088e-09,  3.2858e-08,  1.8656e-08, -1.6938e-08,  1.7084e-08,
        -1.3621e-08, -2.3938e-08,  6.1700e-09], grad_fn=<MeanBackward1>)

Batch Norm의 출력값을 (batch size, width, height에 해당되는 dimension들로) 평균을 내면 대략적으로 평균값이 (0, 0, 0, ... 0)이 되는 것을 확인할 수 있습니다.

### BatchNorm3d 출력값의 표준편차

In [33]:
torch.std(y, axis=[0, 2, 3, 4])

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       grad_fn=<StdBackward0>)

Batch Norm의 출력값을 (batch size, width, height에 해당되는 dimension들로) 표준편차를 내면 대략적으로 표준편차가 (1, 1, 1, ... 1)이 되는 것을 확인할 수 있습니다.

# LayerNorm

In [83]:
batch_size=16
num_features = 8
width = 32
height = 32

In [106]:
# Mean and std different for each data

def data_generator_2d_vary_wrt_data():
    x = torch.normal(0, 1, size=(batch_size, num_features, width, height))
    for i in range(batch_size):
        x[i, :, :, :] = x[i, :, :, :]*(i+1) + i
    
    return x


In [121]:
num_features = 8
width = 32
height = 32

In [122]:
x = data_generator_2d_vary_wrt_data()

### 평균 계산

In [123]:
torch.mean(x, axis=[1, 2, 3])

tensor([-7.9856e-03,  1.0187e+00,  2.0461e+00,  3.0309e+00,  3.9944e+00,
         5.0416e+00,  5.9264e+00,  6.9208e+00,  8.0238e+00,  9.1372e+00,
         1.0176e+01,  1.0890e+01,  1.2004e+01,  1.2870e+01,  1.3913e+01,
         1.5145e+01])

### 표준편차 계산

In [124]:
torch.std(x, axis=[1, 2, 3])

tensor([ 0.9907,  2.0038,  2.9944,  3.9648,  5.0034,  5.9411,  7.0349,  8.0416,
         8.9757,  9.9321, 11.1052, 11.8557, 13.0558, 14.0219, 15.1255, 15.9875])

## LayerNorm Forward pass

In [125]:
norm = nn.LayerNorm(
    normalized_shape=(num_features, width, height),
    elementwise_affine=False
)

In [126]:
y = norm(x)

### LayerNorm 출력값의 평균

In [127]:
torch.mean(y, axis=[1, 2, 3])

tensor([-4.6566e-10,  1.4435e-08,  1.1642e-09, -3.9232e-08, -5.5879e-09,
        -9.2201e-08,  5.2387e-08,  1.2107e-08,  9.7789e-08,  5.0757e-08,
         2.8173e-08,  6.1700e-08, -1.1176e-08, -2.5146e-08, -6.1700e-09,
        -2.3749e-08])

Layer Norm의 출력값을 (channel, width, height에 해당되는 dimension들로) 평균을 내면 대략적으로 평균값이 (0, 0, 0, ... 0)이 되는 것을 확인할 수 있습니다.

### LayerNorm 출력값의 표준편차

In [128]:
torch.std(y, axis=[1, 2, 3])

tensor([1.0001, 1.0001, 1.0001, 1.0001, 1.0001, 1.0001, 1.0001, 1.0001, 1.0001,
        1.0001, 1.0001, 1.0001, 1.0001, 1.0001, 1.0001, 1.0001])

Layer Norm의 출력값을 (channel, width, height에 해당되는 dimension들로) 표준편차를 내면 대략적으로 평균값이 (1, 1, 1, ... 1)이 되는 것을 확인할 수 있습니다.

### 동일한 예시에 BatchNorm을 적용해보자

In [129]:
norm = nn.BatchNorm2d(num_features=8,
                     affine=True)  ## BatchNorm2d Forward pass

In [130]:
y = norm(x)

In [131]:
torch.mean(y, axis=[1, 2, 3])

tensor([-0.7013, -0.6055, -0.5097, -0.4177, -0.3279, -0.2302, -0.1476, -0.0548,
         0.0481,  0.1521,  0.2489,  0.3157,  0.4194,  0.5001,  0.5977,  0.7125],
       grad_fn=<MeanBackward1>)

Batch Norm의 출력값을 (channel, width, height에 해당되는 dimension들로) 평균을 내면 대략적으로 평균값이 (0, 0, 0, ... 0)이 되지 않습니다!

In [132]:
torch.std(y, axis=[1, 2, 3])

tensor([0.0928, 0.1871, 0.2795, 0.3700, 0.4668, 0.5543, 0.6564, 0.7504, 0.8373,
        0.9266, 1.0362, 1.1061, 1.2180, 1.3081, 1.4115, 1.4914],
       grad_fn=<StdBackward0>)

Batch Norm의 출력값을 (channel, width, height에 해당되는 dimension들로) 표준편차를 내면 대략적으로 평균값이 (1, 1, 1, ... 1)이 되지 않습니다!

# InstanceNorm

In [48]:
# mean, std vary wrt batch and channel
def data_generator_2d_vary_wrt_data_channel():
    x = torch.normal(0, 1, size=(batch_size, num_features, width, height))
    count = 0
    for i in range(batch_size):
        for j in range(num_features):
            x[i, j, :, :] = x[i, j, :, :]*(count+1) + count
            count += 1
    
    return x



InstanceNorm의 경우 Height, Width, Batch을 고정했을때, Channel의 axis 방향으로 평균과 표준편차가 일정하도록 normalize해줍니다.

In [133]:
batch_size=16
num_features = 8
width = 32
height = 32

In [134]:
x = data_generator_2d_vary_wrt_data_channel()

### 평균 계산

In [135]:
torch.mean(torch.mean(x, axis=[1]))

tensor(63.2102)

### 표준편차 계산

In [136]:
torch.mean(torch.std(x, axis=[1]))

tensor(62.3172)

## InstanceNorm Forward pass

In [137]:
norm = nn.InstanceNorm2d(
    num_features=num_features,
    affine=True
)

In [138]:
y = norm(x)

### InstanceNorm 출력값의 평균

In [139]:
torch.mean(torch.mean(y, axis=[1]))

tensor(-8.7311e-10, grad_fn=<MeanBackward0>)

Instance Norm의 출력값을 (width, height에 해당되는 dimension들로) 평균을 내면 각 (batch, channel)에 대해서 대략적으로 평균값이 (0, 0, 0, ... 0)이 되는 것을 확인할 수 있습니다.

### InstanceNorm 출력값의 표준편차

In [141]:
torch.mean(torch.std(y, axis=[1]))

tensor(0.9638, grad_fn=<MeanBackward0>)

Instance Norm의 출력값을 (width, height에 해당되는 dimension들로) 표준편차를 내면 각 (batch, channel)에 대해서 대략적으로 표준편차값이 (1, 1, 1, ... 1)이 되는 것을 확인할 수 있습니다.